In [ ]:
using JuMP, Gurobi, DrakeVisualizer, GeometryTypes, CoordinateTransformations
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()



## Model-Predictive Control in JuMP

Let's implement a very simple model-predictive control (MPC) optimization in JuMP. Specifically, we'll write an optimization that tries to find a sequence of control inputs for a very simple model robot in order to optimize an objective function. 

More concretely, our model will be a brick, sliding frictionlessly in two dimensions. Our input will be the acceleration of the brick, and we'll try to minimize the brick's final velocity and distance from the origin. 

In [ ]:
Δt = 0.1
num_time_steps = 10
max_acceleration = 0.5

# run_mpc() takes the robot's current position and velocity
# and returns an optimized trajectory of position, velocity, 
# and acceleration. 
function run_mpc(initial_position, initial_velocity)
    
    model = Model(solver=GurobiSolver(OutputFlag=0))

    @variables model begin
        position[1:2, 1:num_time_steps]
        velocity[1:2, 1:num_time_steps]
        -max_acceleration <= acceleration[1:2, 1:num_time_steps] <= max_acceleration
    end

    # Dynamics constraints
    @constraint(model, [i=2:num_time_steps, j=1:2],
                velocity[j, i] == velocity[j, i - 1] + acceleration[j, i - 1] * Δt)
    @constraint(model, [i=2:num_time_steps, j=1:2],
                position[j, i] == position[j, i - 1] + velocity[j, i - 1] * Δt)

    # Cost function: minimize final position and final velocity
    @objective(model, Min, 
        10 * sum(position[:, end].^2) + sum(velocity[:, end].^2))

    # Initial conditions:
    @constraint(model, position[:, 1] .== initial_position)
    @constraint(model, velocity[:, 1] .== initial_velocity)

    solve(model)
    return getvalue(position), getvalue(velocity), getvalue(acceleration)
end

# A convenience function for getting the individual positions at each time
columns(x::AbstractMatrix) = [x[:, i] for i in 1:size(x, 2)]

position, velocity, acceleration = run_mpc([2, 0], [-1, 0])

# Visualizing the Result

We'll show the result of our optimization using the [DrakeVisualizer.jl](https://github.com/rdeits/DrakeVisualizer.jl) package. To do that, we'll define a `Visualizer` and create some geometries to represent the (box) robot and its goal:

In [ ]:
vis = Visualizer()[:mpc]
setgeometry!(vis[:goal], HyperSphere(Point(0., 0, 0), 0.1))
setgeometry!(vis[:block], HyperRectangle(Vec(-0.2, -0.2, -0.2), Vec(0.4, 0.4, 0.4)));

The draw() function will take a sequence of robot positions
and draw the current pose of the block as well as an arrow
showing the planned trajectory from the MPC optimization

In [ ]:
function draw(vis, positions)
    settransform!(vis[:block], Translation(vcat(positions[:, 1], 0)))
    setgeometry!(vis[:horizon], 
        PolyLine([vcat(pos, 0) for pos in columns(positions)],
            radius=0.02,
            end_head=ArrowHead(0.08)))
end

In [ ]:
draw(vis, position);

In [ ]:
for i in 1:size(position, 2)
    draw(vis, position[:, i:end])
    sleep(Δt)
end

## Running the MPC Controller

In a real application, we wouldn't just run the MPC optimization once. Instead, we might run the optimization at every time step using the robot's current state. 

To do that, let's wrap the MPC problem in a function called `run_mpc()` that takes the robot's current position and velocity as input:

We can demonstrate this by repeatedly running the MPC program, applying its planned acceleration to the brick, and then simulating one step forward in time:

In [ ]:
# The robot's starting position and velocity
q = randn(2)
v = randn(2)

while (norm(q) > 0.2 || norm(v) > 0.2)
    
    # Run the MPC control optimization
    q_plan, v_plan, u_plan = run_mpc(q, v)
    
    # Draw the planned future states from the MPC optimization
    draw(vis, q_plan)
    
    # Apply the planned acceleration and simulate one step in time
    u = u_plan[:, 1]
    v += u * Δt
    q += v * Δt
    sleep(Δt)
end